In [ ]:
%config Completer.use_jedi = False

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
%matplotlib inline

In [ ]:
from pymongo import MongoClient
client = MongoClient()
db = client.Project

In [ ]:
collection = db.Users
OriginalData = pd.DataFrame(list(collection.find()))

In [ ]:
collection = db.Fusers
FakeData = pd.DataFrame(list(collection.find()))

In [ ]:
OriginalData.drop("_id",axis=1,inplace=True)

In [ ]:
FakeData.drop("_id",axis=1,inplace=True)

In [ ]:
gusers=OriginalData
fusers=FakeData

In [ ]:
x=pd.concat([gusers,fusers])

In [ ]:
y=len(gusers)*[1]+len(fusers)*[0]

In [ ]:
x

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,created_at,url,...,profile_sidebar_fill_color,profile_background_image_url,profile_background_color,profile_link_color,utc_offset,protected,verified,description,updated,dataset
0,3610511,Davide Dellacasa,braddd,20370,5470,2385,145,52,Fri Apr 06 10:58:22 +0000 2007,http://braddd.tumblr.com,...,FFF7CC,http://a0.twimg.com/profile_background_images/...,BADFCD,FF0000,3600,,,Founder of http://www.screenweek.it & http://w...,2015-02-14 10:54:49,E13
1,5682702,tacone,tacone_,4024,264,87,323,16,Tue May 01 11:53:40 +0000 2007,http://t.co/LKrl1dZE,...,0,http://a0.twimg.com/profile_background_images/...,1A1B1F,2FC2EF,3600,,,Cogito ergo bestemmio.,2015-02-14 10:54:49,E13
2,6140012,CRISTIAN PAOLINI,crispaol,3603,138,179,53,1,Fri May 18 15:16:20 +0000 2007,,...,DAECF4,http://a0.twimg.com/profile_background_images/...,C6E2EE,1F98C7,3600,,,"Swiper non rubare, Swiper non rubare!",2015-02-14 10:54:49,E13
3,6134312,tmpx,tmpx,1183,128,168,2,5,Fri May 18 10:28:11 +0000 2007,,...,EFEFEF,http://a0.twimg.com/images/themes/theme14/bg.gif,131516,9999,3600,,,I never saw a wild thing sorry for itself.A sm...,2015-02-14 10:54:49,E13
4,5656162,Simone Economo,eKoeS,3131,506,381,9,40,Mon Apr 30 15:08:42 +0000 2007,http://www.lineheight.net/,...,DDEEF6,http://a0.twimg.com/images/themes/theme1/bg.png,C0DEED,0084B4,3600,,,BSc degree (cum laude) in Computer Engineering...,2015-02-14 10:54:49,E13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,1391832212,Harriett Harvey,HarriettHarvey9,2,0,16,0,0,Tue Apr 30 11:25:11 +0000 2013,,...,DDEEF6,http://a0.twimg.com/images/themes/theme1/bg.png,C0DEED,0084B4,,,,,2015-02-14 10:40:01,INT
1333,1388028872,Delmer Morris,DelmerMorris1,1,1,12,0,0,Sun Apr 28 22:04:02 +0000 2013,,...,DDEEF6,http://a0.twimg.com/images/themes/theme1/bg.png,C0DEED,0084B4,,,,"| TeamDarkSkinn , 5'9 , Tha Name iss BiiG RICH...",2015-02-14 10:40:01,INT
1334,1391544607,Danial Campbell,DanialCampbell2,0,1,17,0,0,Tue Apr 30 08:34:49 +0000 2013,,...,DDEEF6,http://a0.twimg.com/images/themes/theme1/bg.png,C0DEED,0084B4,,,,,2015-02-14 10:40:01,INT
1335,1391998039,Gillian Wheeler,GillianWheeler3,0,0,17,0,0,Tue Apr 30 12:47:51 +0000 2013,,...,DDEEF6,http://a0.twimg.com/images/themes/theme1/bg.png,C0DEED,0084B4,,,,Hello!im a BELIEBER! IM A HUGE FAN OF JUSTIN B...,2015-02-14 10:40:01,INT


In [ ]:
def extract_features(x):
    lang_list = list(enumerate(np.unique(x['lang'])))   
    lang_dict = { name : i for i, name in lang_list }             
    x.loc[:,'lang_code'] = x['lang'].map( lambda x: lang_dict[x]).astype(int)    
    feature_columns_to_use = ['statuses_count','followers_count','friends_count','favourites_count','listed_count','lang_code']
    x=x.loc[:,feature_columns_to_use]
    return x
  
x=extract_features(x)

In [ ]:
x

,statuses_count,followers_count,friends_count,favourites_count,listed_count,lang_code
0,20370,5470,2385,145,52,5
1,4024,264,87,323,16,1
2,3603,138,179,53,1,5
3,1183,128,168,2,5,1
4,3131,506,381,9,40,1
...,...,...,...,...,...,...
1332,2,0,16,0,0,1
1333,1,1,12,0,0,1
1334,0,1,17,0,0,1
1335,0,0,17,0,0,1


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(x, y, test_size=0.20, random_state=44)

## SVM

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
mm=MinMaxScaler()
x_train=mm.fit_transform(X_train)
x_test=mm.transform(X_test)

In [ ]:
model=SVC()
model.fit(x_train,y_train)

SVC()

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
type(y_pred)

numpy.ndarray

In [ ]:
accuracy_score(y_test, y_pred)

0.9698581560283688

In [ ]:
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[267   0]
 [ 17 280]]


In [ ]:
print(classification_report(y_test, y_pred, target_names=['Fake','Genuine']))

              precision    recall  f1-score   support

        Fake       0.94      1.00      0.97       267
     Genuine       1.00      0.94      0.97       297

    accuracy                           0.97       564
   macro avg       0.97      0.97      0.97       564
weighted avg       0.97      0.97      0.97       564



## LR

In [ ]:
model=LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
prediction=model.predict(X_test)

In [ ]:
accuracy_score(y_test, prediction)

0.9946808510638298

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,prediction))

[[267   0]
 [  3 294]]


In [ ]:
print(classification_report(y_test, prediction, target_names=['Fake','Genuine']))

              precision    recall  f1-score   support

        Fake       0.99      1.00      0.99       267
     Genuine       1.00      0.99      0.99       297

    accuracy                           0.99       564
   macro avg       0.99      0.99      0.99       564
weighted avg       0.99      0.99      0.99       564

